### Import Library

In [1]:
import pandas as pd
import numpy as np
from linearmodels import PanelOLS,FamaMacBeth

In [2]:
import statsmodels.api as sm

### Import Data

In [4]:
data_directory = "data/"
file = "yahoo_data_filter_0_0_1.csv"

In [5]:
df = pd.read_csv(data_directory + file)
df["date"] = pd.to_datetime(df["date"])
df = df.set_index("date")

### Calculate the momentun signal

In [11]:
universe_df = df

In [12]:
momentum_period = 5
waiting_time = 1

In [13]:
complete_ret_sub_df = universe_df.pct_change(periods=momentum_period).shift(waiting_time).dropna()

In [14]:
complete_ret_sub_df_long = complete_ret_sub_df.stack()

### Create the future return

In [15]:
future_return_days = 1

In [16]:
future_return_df = universe_df.pct_change(future_return_days).shift(-(future_return_days-1))

In [17]:
future_return_df_long = future_return_df.stack()

### Merge Return with Signal

In [18]:
future_return_df_long = pd.DataFrame(future_return_df_long)
complete_ret_sub_df_long = pd.DataFrame(complete_ret_sub_df_long)

In [19]:
future_return_df_long.columns = ["target"]
complete_ret_sub_df_long.columns = ["signal"]

In [20]:
merge_df = future_return_df_long.merge(complete_ret_sub_df_long,left_index=True,right_index=True)

In [21]:
index_df = merge_df.index.to_frame()
index_df.columns = ["date","asset"]
index_df["target"] = merge_df["target"]
index_df["signal"] = merge_df["signal"]
index_df.index = range(0,index_df.shape[0])

In [22]:
merge_df = index_df

In [23]:
merge_df = merge_df.set_index(["asset","date"])

### Panel Regression

In [24]:
exog_vars = ["signal"]
depe_var = ["target"]
exog = sm.add_constant(merge_df[exog_vars])
depe = merge_df[depe_var]

In [25]:
mod = PanelOLS(depe,exog,entity_effects=True,time_effects=True)

In [26]:
mod.fit(cov_type="clustered",cluster_entity=True,cluster_time=True)

C:\app_user_download\anaconda\lib\site-packages\linearmodels\panel\results.py:84: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")


Dep. Variable:,target,R-squared:,0.0028
Estimator:,PanelOLS,R-squared (Between):,-0.2396
No. Observations:,80734,R-squared (Within):,-0.0003
Date:,"Wed, Jan 20 2021",R-squared (Overall):,-0.0004
Time:,23:52:25,Log-likelihood,1.237e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,226.73
Entities:,74,P-value,0.0000
Avg Obs:,1091.0,Distribution:,"F(1,79569)"
Min Obs:,1091.0,,
Max Obs:,1091.0,F-statistic (robust):,9.1725


In [27]:
mod = PanelOLS(depe,exog,entity_effects=True,time_effects=True)

In [28]:
mod.fit(cov_type="kernel",kernel="parzen",cluster_entity=True,cluster_time=True)

Dep. Variable:,target,R-squared:,0.0028
Estimator:,PanelOLS,R-squared (Between):,-0.2396
No. Observations:,80734,R-squared (Within):,-0.0003
Date:,"Wed, Jan 20 2021",R-squared (Overall):,-0.0004
Time:,23:52:35,Log-likelihood,1.237e+05
Cov. Estimator:,Driscoll-Kraay,,
,,F-statistic:,226.73
Entities:,74,P-value,0.0000
Avg Obs:,1091.0,Distribution:,"F(1,79569)"
Min Obs:,1091.0,,
Max Obs:,1091.0,F-statistic (robust):,17.137


### Fama Macbeth Regression

In [29]:
exog_vars = ["signal"]
depe_var = ["target"]
exog = sm.add_constant(merge_df[exog_vars])
depe = merge_df[depe_var]

In [30]:
mod = FamaMacBeth(depe,exog)

In [31]:
res = mod.fit(cov_type='kernel',kernel="bartlett")

In [32]:
res

Dep. Variable:,target,R-squared:,-6.967e-05
Estimator:,FamaMacBeth,R-squared (Between):,-0.4612
No. Observations:,80734,R-squared (Within):,6.389e-05
Date:,"Wed, Jan 20 2021",R-squared (Overall):,-6.967e-05
Time:,23:52:44,Log-likelihood,1.008e+05
Cov. Estimator:,Fama-MacBeth Kernel Cov,,
,,F-statistic:,-5.6243
Entities:,74,P-value,1.0000
Avg Obs:,1091.0,Distribution:,"F(1,80732)"
Min Obs:,1091.0,,
Max Obs:,1091.0,F-statistic (robust):,21.511


### Multiple Signal Panel Regression

#### Create Multiple Signal 

In [33]:
momentum_period = 1
waiting_time = 1

In [34]:
complete_ret_sub_df_s1 = universe_df.pct_change(periods=momentum_period).shift(waiting_time).dropna()
complete_ret_sub_df_s1_long = complete_ret_sub_df_s1.stack()

In [35]:
momentum_period = 3
waiting_time = 1

In [36]:
complete_ret_sub_df_s3 = universe_df.pct_change(periods=momentum_period).shift(waiting_time).dropna()
complete_ret_sub_df_s3_long = complete_ret_sub_df_s3.stack()

In [37]:
momentum_period = 5
waiting_time = 1

In [38]:
complete_ret_sub_df_s5 = universe_df.pct_change(periods=momentum_period).shift(waiting_time).dropna()
complete_ret_sub_df_s5_long = complete_ret_sub_df_s5.stack()

In [39]:
momentum_period = 7
waiting_time = 1

In [40]:
complete_ret_sub_df_s7 = universe_df.pct_change(periods=momentum_period).shift(waiting_time).dropna()
complete_ret_sub_df_s7_long = complete_ret_sub_df_s7.stack()

#### Create Future Return

In [41]:
future_return_days = 1

future_return_df = universe_df.pct_change(future_return_days).shift(-(future_return_days-1))
future_return_df_long = future_return_df.stack()

future_return_df_long = pd.DataFrame(future_return_df_long)

#### Merge the dataframe

In [42]:
future_return_df_long = pd.DataFrame(future_return_df_long)

In [43]:
signal_df = pd.concat([complete_ret_sub_df_s1_long,
                       complete_ret_sub_df_s3_long,
                       complete_ret_sub_df_s5_long,
                       complete_ret_sub_df_s7_long],axis=1)

In [44]:
merge_df = future_return_df_long.merge(signal_df,left_index=True,right_index=True)

In [45]:
merge_df.columns = ["target","s1","s3","s5","s7"]



index_df = merge_df.index.to_frame()
index_df.columns = ["date","asset"]


index_df["target"] = merge_df["target"]
index_df["s1"] = merge_df["s1"]
index_df["s3"] = merge_df["s3"]
index_df["s5"] = merge_df["s5"]
index_df["s7"] = merge_df["s7"]

index_df.index = range(0,index_df.shape[0])

In [46]:
merge_df = index_df

merge_df = merge_df.set_index(["asset","date"])
merge_df = merge_df.dropna()

#### Panel Regression

In [47]:
exog_vars = ["s1","s3","s5","s7"]
depe_var = ["target"]
exog = sm.add_constant(merge_df[exog_vars])
depe = merge_df[depe_var]

In [48]:
mod = PanelOLS(depe,exog,entity_effects=True,time_effects=True)
mod.fit(cov_type="clustered",cluster_entity=True,cluster_time=True)

C:\app_user_download\anaconda\lib\site-packages\linearmodels\panel\results.py:84: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")


Dep. Variable:,target,R-squared:,0.0102
Estimator:,PanelOLS,R-squared (Between):,-0.2441
No. Observations:,80586,R-squared (Within):,0.0077
Date:,"Wed, Jan 20 2021",R-squared (Overall):,0.0076
Time:,23:53:04,Log-likelihood,1.243e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,205.05
Entities:,74,P-value,0.0000
Avg Obs:,1089.0,Distribution:,"F(4,79420)"
Min Obs:,1089.0,,
Max Obs:,1089.0,F-statistic (robust):,7.7276


In [49]:
mod = PanelOLS(depe,exog,entity_effects=False,time_effects=False)

In [50]:
mod.fit(cov_type="clustered",cluster_entity=True,cluster_time=True)

Dep. Variable:,target,R-squared:,0.0083
Estimator:,PanelOLS,R-squared (Between):,-0.1325
No. Observations:,80586,R-squared (Within):,0.0083
Date:,"Wed, Jan 20 2021",R-squared (Overall):,0.0083
Time:,23:53:53,Log-likelihood,1.013e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,168.15
Entities:,74,P-value,0.0000
Avg Obs:,1089.0,Distribution:,"F(4,80581)"
Min Obs:,1089.0,,
Max Obs:,1089.0,F-statistic (robust):,4.1737


#### Multiple Fama Macbeth Regression

In [51]:
mod = FamaMacBeth(depe,exog)

In [52]:
res = mod.fit(cov_type='kernel',kernel="bartlett")

In [53]:
res

Dep. Variable:,target,R-squared:,0.0073
Estimator:,FamaMacBeth,R-squared (Between):,-1.3296
No. Observations:,80586,R-squared (Within):,0.0077
Date:,"Wed, Jan 20 2021",R-squared (Overall):,0.0073
Time:,23:54:00,Log-likelihood,1.013e+05
Cov. Estimator:,Fama-MacBeth Kernel Cov,,
,,F-statistic:,148.77
Entities:,74,P-value,0.0000
Avg Obs:,1089.0,Distribution:,"F(4,80581)"
Min Obs:,1089.0,,
Max Obs:,1089.0,F-statistic (robust):,20.683


### Multiple Holding Days

In [54]:
future_return_days = 2

future_return_df = universe_df.pct_change(future_return_days).shift(-(future_return_days-1))
future_return_df_long = future_return_df.stack()

future_return_df_long = pd.DataFrame(future_return_df_long)

In [55]:
merge_df = future_return_df_long.merge(signal_df,left_index=True,right_index=True)

In [56]:
merge_df

0_x       0_y         1         2         3
date                                                                 
2018-01-02 ADA-USD   0.481712  0.012453       NaN       NaN       NaN
           ADX-USD  -0.080739  0.114517       NaN       NaN       NaN
           AE-USD    0.175440 -0.015999       NaN       NaN       NaN
           ARDR-USD  0.215451 -0.065361       NaN       NaN       NaN
           ARK-USD  -0.001632  0.080079       NaN       NaN       NaN
...                       ...       ...       ...       ...       ...
2021-01-02 XVG-USD   0.417309 -0.022395 -0.015002 -0.065067  0.100306
           XZC-USD  -0.045698 -0.094004 -0.056278 -0.142921 -0.056563
           ZEC-USD   0.029607 -0.112465 -0.146453 -0.160452 -0.086150
           ZEN-USD   0.182516  0.018508  0.018827 -0.059865  0.050730
           ZRX-USD   0.019493  0.058472  0.026020  0.004323  0.046617

[80956 rows x 5 columns]

In [57]:
merge_df

0_x       0_y         1         2         3
date                                                                 
2018-01-02 ADA-USD   0.481712  0.012453       NaN       NaN       NaN
           ADX-USD  -0.080739  0.114517       NaN       NaN       NaN
           AE-USD    0.175440 -0.015999       NaN       NaN       NaN
           ARDR-USD  0.215451 -0.065361       NaN       NaN       NaN
           ARK-USD  -0.001632  0.080079       NaN       NaN       NaN
...                       ...       ...       ...       ...       ...
2021-01-02 XVG-USD   0.417309 -0.022395 -0.015002 -0.065067  0.100306
           XZC-USD  -0.045698 -0.094004 -0.056278 -0.142921 -0.056563
           ZEC-USD   0.029607 -0.112465 -0.146453 -0.160452 -0.086150
           ZEN-USD   0.182516  0.018508  0.018827 -0.059865  0.050730
           ZRX-USD   0.019493  0.058472  0.026020  0.004323  0.046617

[80956 rows x 5 columns]

In [58]:
merge_df.columns = ["target","s1","s3","s5","s7"]



index_df = merge_df.index.to_frame()
index_df.columns = ["date","asset"]


index_df["target"] = merge_df["target"]
index_df["s1"] = merge_df["s1"]
index_df["s3"] = merge_df["s3"]
index_df["s5"] = merge_df["s5"]
index_df["s7"] = merge_df["s7"]

index_df.index = range(0,index_df.shape[0])

In [59]:
merge_df = index_df

merge_df = merge_df.set_index(["asset","date"])
merge_df = merge_df.dropna()

#### Panel Regression

In [60]:
exog_vars = ["s1","s3","s5","s7"]
depe_var = ["target"]
exog = sm.add_constant(merge_df[exog_vars])
depe = merge_df[depe_var]

In [61]:
mod = PanelOLS(depe,exog,entity_effects=True,time_effects=True)
mod.fit(cov_type="clustered",cluster_entity=True,cluster_time=True)

C:\app_user_download\anaconda\lib\site-packages\linearmodels\panel\results.py:84: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")


Dep. Variable:,target,R-squared:,0.0085
Estimator:,PanelOLS,R-squared (Between):,-0.1229
No. Observations:,80512,R-squared (Within):,0.0026
Date:,"Wed, Jan 20 2021",R-squared (Overall):,0.0025
Time:,23:54:15,Log-likelihood,9.963e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,169.12
Entities:,74,P-value,0.0000
Avg Obs:,1088.0,Distribution:,"F(4,79347)"
Min Obs:,1088.0,,
Max Obs:,1088.0,F-statistic (robust):,10.388


In [62]:
mod = PanelOLS(depe,exog,entity_effects=False,time_effects=False)
mod.fit(cov_type="clustered",cluster_entity=False,cluster_time=True)

Dep. Variable:,target,R-squared:,0.0046
Estimator:,PanelOLS,R-squared (Between):,-0.0164
No. Observations:,80512,R-squared (Within):,0.0046
Date:,"Wed, Jan 20 2021",R-squared (Overall):,0.0046
Time:,23:54:16,Log-likelihood,7.639e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,92.526
Entities:,74,P-value,0.0000
Avg Obs:,1088.0,Distribution:,"F(4,80507)"
Min Obs:,1088.0,,
Max Obs:,1088.0,F-statistic (robust):,4.4062


In [63]:
mod = PanelOLS(depe,exog,entity_effects=False,time_effects=False)
mod.fit(cov_type="clustered",cluster_entity=True,cluster_time=True)

Dep. Variable:,target,R-squared:,0.0046
Estimator:,PanelOLS,R-squared (Between):,-0.0164
No. Observations:,80512,R-squared (Within):,0.0046
Date:,"Wed, Jan 20 2021",R-squared (Overall):,0.0046
Time:,23:54:16,Log-likelihood,7.639e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,92.526
Entities:,74,P-value,0.0000
Avg Obs:,1088.0,Distribution:,"F(4,80507)"
Min Obs:,1088.0,,
Max Obs:,1088.0,F-statistic (robust):,3.7952


#### Fama Macbeth

In [64]:
mod = FamaMacBeth(depe,exog)

In [65]:
mod.fit(cov_type='kernel',kernel="bartlett")

Dep. Variable:,target,R-squared:,0.0031
Estimator:,FamaMacBeth,R-squared (Between):,-0.7816
No. Observations:,80512,R-squared (Within):,0.0035
Date:,"Wed, Jan 20 2021",R-squared (Overall):,0.0031
Time:,23:54:18,Log-likelihood,7.633e+04
Cov. Estimator:,Fama-MacBeth Kernel Cov,,
,,F-statistic:,61.583
Entities:,74,P-value,0.0000
Avg Obs:,1088.0,Distribution:,"F(4,80507)"
Min Obs:,1088.0,,
Max Obs:,1088.0,F-statistic (robust):,13.208
